In [1]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00


In [3]:
import openai

openai.api_key = your_key

In [4]:
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613",openai_api_key = openai.api_key)

In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Map
map_template = """List all the technical words in the text provided. Do not add any other information
{docs}"
"""
map_prompt = PromptTemplate.from_template(map_template)
print(map_prompt)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

input_variables=['docs'] output_parser=None partial_variables={} template='List all the technical words in the text provided. Do not add any other information\n{docs}"\n' template_format='f-string' validate_template=True


In [27]:
from langchain.chains import StuffDocumentsChain, ReduceDocumentsChain, MapReduceDocumentsChain

# Reduce
reduce_template = """The following is a list of keywords:
{doc_keywords}
Take these and distill it into a final, consolidated list of keywords in alphabetical order.
"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="doc_keywords"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [28]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)


In [29]:
!pip install pypdf

In [30]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("3.SLM Foundation of Social Thought.pdf")
data = loader.load()

print(len(data))
print(len(data[100].page_content))

148
2305


In [31]:
print(data[100].page_content)
text = data[100].page_content

Socialization and  Social  Control  Module  97  
 under the direct control of the government who take care of his/her population  
and trying  to provide  basic  essentials  with security.  
In modern  times,  there  are different  types  of government  with varied  
philosophies like democracy, communism, dictatorship and others. In modern  
societies  government  make  rules  and regulation  which  implemented  and 
guarded  with the help of given  constitution  of a particular  country  or state.  
12. Culture: Culture is a very extensive term; it simply defined as the manmade  
part of our surroundings. It helps to influencing the nature and social behaviour  
of an individual  in present  society.  
3.2.3.  Conformity  
 
Introduction  
Conformity  studies  stresses  on persons  to conform  the prospects  of a group,  
association, institute, society or leader. It is a category of social effect comprising a  
variation in trust or actions in command to fitting in group. It may inc

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
#all_splits = text_splitter.split_documents(docs)

In [33]:
# For checking
# Create Document objects for each text chunk
from langchain.docstore.document import Document

texts = text_splitter.split_text( data[100].page_content)
docs = [Document(page_content=t) for t in texts[:]]

In [34]:
print(len(docs))

3


In [35]:
!pip install tiktoken

In [36]:
print(map_reduce_chain.run(docs))

- Actions
- Akshay Kumar
- Alia Bhatt
- Association
- Attitude
- Beliefs
- Celebrity
- Change
- Command
- Compliance
- Conformity
- Constitution
- Culture
- Democracy
- Dictatorship
- Encouraging
- Essentials
- Feeling
- Fig. 3.2.3
- Fitting
- Follow
- Friends
- Government
- Group
- Harvard psychologists
- Hope
- Identification
- Individual
- Influence
- Influencing
- Informal
- Institute
- Internalisation
- Kangana Ranaut
- Leader
- Liking
- Linking
- Major
- Manmade
- Modern
- Nature
- Necessary
- Norms
- Open
- Panic
- Persons
- Philosophies
- Physical
- Population
- Pressure
- Rejected
- Regulation
- Respected
- Rules
- Security
- Social behavior
- Social control
- Social crowds
- Social effect
- Social norms
- Society
- Studies
- Surroundings
- Trust
- Types
- Views
